<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Remediate unencrypted S3 buckets using unSkript legos.</b>
</div>

<br>

<center><h2>Remediate unencrypted S3 buckets</h2></center>

# Steps Overview
   1) Filter all the S3 buckets which are unencrypted.
   2) Apply encryption on unencrypted S3 buckets.

Here we will use unSkript Filter Unencrypted S3 Buckets Lego. This lego takes region: str as input. This inputs is used to filter all the S3 buckets which don't have encryption settings.

In [5]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from botocore.exceptions import ClientError
import pprint

from beartype import beartype
@beartype
def aws_filter_unencrypted_s3_buckets_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_filter_unencrypted_s3_buckets(handle, region: str) -> List:
    """aws_filter_unencrypted_s3_buckets List of unencrypted bucket name .

          :type region: string
          :param region: Filter S3 buckets.

          :rtype: List with unencrypted bucket name.
      """
    s3Client = handle.client('s3',
                             region_name=region)

    response = s3Client.list_buckets()

    # List unencrypted S3 buckets
    result = []
    for bucket in response['Buckets']:
        try:
            response = s3Client.get_bucket_encryption(Bucket=bucket['Name'])
            encRules = response['ServerSideEncryptionConfiguration']['Rules']

        except ClientError as e:
            result.append(bucket['Name'])

    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
}''')
task.configure(inputParamsJson='''{
    "region": "\\"us-west-2\\""
    }''')
task.configure(outputName="Bucket_list")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_unencrypted_s3_buckets, lego_printer=aws_filter_unencrypted_s3_buckets_printer, hdl=hdl, args=args)

Here we will use unSkript Apply AWS Default Encryption for S3 Buckets Lego. This lego takes name: str, region: str as input. This inputs is used to apply the default encryption configuration to the unencrypted S3 buckets.

In [4]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_put_bucket_encryption_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_put_bucket_encryption(handle, name: str, region: str) -> Dict:
    """aws_put_bucket_encryption Puts default encryption configuration for bucket.

          :type name: string
          :param name: NAme of the S3 bucket.

          :type region: string
          :param region: location of the bucket

          :rtype: Dict with the response info.
      """
    s3Client = handle.client('s3',
                             region_name=region)

    # Setup default encryption configuration 
    response = s3Client.put_bucket_encryption(
        Bucket=name,
        ServerSideEncryptionConfiguration={
            "Rules": [
                {"ApplyServerSideEncryptionByDefault": {"SSEAlgorithm": "AES256"}}
            ]},
        )
    return response


task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(credentialsJson='''{
}''')
task.configure(inputParamsJson='''{
    "region": "\\"us-west-2\\"",
    "name": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "Bucket_list",
    "iter_parameter": "name"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(Bucket_list) > 0",
    "condition_result": true
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_put_bucket_encryption, lego_printer=aws_put_bucket_encryption_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to filter all unencrypted S3 buckets and apply default encryption configuration to the buckets. To view the full platform capabilities of unSkript please visit https://unskript.com